# geDIG Attention Update (Modern LLMs)

Goal: sample attention maps from a newer model (e.g., Llama 3 / Phi-3) and compute geDIG F to confirm whether the sign and delta vs. random are preserved.

Notes:
- Llama 3 requires a Hugging Face token and license acceptance.
- If you do not have access, use an open model such as `microsoft/Phi-3-mini-4k-instruct`.


In [ ]:
from google.colab import drive

drive.mount('/content/drive')


In [ ]:
!git clone https://github.com/miyauchikazuyoshi/InsightSpike-AI.git
%cd InsightSpike-AI
!pip -q install -U pip
!pip -q install transformers datasets accelerate sentencepiece networkx scipy


In [ ]:
from huggingface_hub import login

# login()  # Uncomment and paste your HF token for gated models


In [ ]:
MODEL_ID = "meta-llama/Meta-Llama-3-8B"  # requires HF token + license
# MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"  # open fallback
DTYPE = "bfloat16"  # use "float16" for T4, "float32" for CPU
OUT = "results/transformer_gedig/score_llm_update.json"

!python experiments/transformer_gedig/extract_and_score.py \
  --model "$MODEL_ID" \
  --text-count 16 \
  --layer-cap 4 \
  --attn-max-len 256 \
  --percentile 0.90 \
  --out "$OUT" \
  --device auto \
  --dtype "$DTYPE" \
  --device-map auto \
  --trust-remote-code


In [ ]:
import json
import numpy as np

with open(OUT, "r", encoding="utf-8") as fh:
    data = json.load(fh)

f_real = np.mean([d["F"] for d in data])
f_rand = np.mean([d["baseline_F_random"] for d in data])
print("F_real mean:", f_real)
print("F_random mean:", f_rand)
print("Delta (real - random):", f_real - f_rand)


In [ ]:
from pathlib import Path
import shutil

out_path = Path(OUT)
dest_dir = Path("/content/drive/MyDrive/insightspike/gedig_attention")
dest_dir.mkdir(parents=True, exist_ok=True)
shutil.copy2(out_path, dest_dir / out_path.name)
print("Saved to:", dest_dir / out_path.name)
